<a href="https://colab.research.google.com/github/Barry-ZhangUofT/Parameter-optimization/blob/main/Grid_search_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Helper Function

In [1]:
try:
    # Check if notebook is running in Google Colab
    import google.colab
    # Get additional files from Github
    !wget https://raw.githubusercontent.com/krasserm/bayesian-machine-learning/master/bayesian_optimization_util.py
    # Install additional dependencies
    !pip install scikit-optimize==0.5.2
    !pip install GPy==1.9.8
    !pip install GPyOpt==1.2.1
    !pip install xgboost==0.90
    !pip install --upgrade scipy==1.4.1


except:
    pass

--2024-03-22 18:39:11--  https://raw.githubusercontent.com/krasserm/bayesian-machine-learning/master/bayesian_optimization_util.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1536 (1.5K) [text/plain]
Saving to: ‘bayesian_optimization_util.py.1’

bayesian_optimizati 100%[===================>]   1.50K  --.-KB/s    in 0s      

2024-03-22 18:39:11 (24.3 MB/s) - ‘bayesian_optimization_util.py.1’ saved [1536/1536]

  Using cached GPyOpt-1.2.1.tar.gz (46 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed


In [2]:
!pip install scikit-learn
!pip install numpy

In [3]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
from bayesian_optimization_util import plot_approximation, plot_acquisition
from sklearn.gaussian_process.kernels import RBF,ConstantKernel as C,RationalQuadratic as RQ,WhiteKernel as WK,ExpSineSquared as Exp,DotProduct as Lin
from numpy import exp,arange
from pylab import meshgrid,cm,imshow,contour,clabel,colorbar,axis,title,show

from mlxtend.preprocessing import standardize,minmax_scaling

import pandas as pd

from sklearn.model_selection import train_test_split, RepeatedKFold
import time
from IPython.display import clear_output
from operator import add

from sklearn.metrics import mean_absolute_error, mean_squared_log_error, r2_score, median_absolute_error


from sklearn.preprocessing import minmax_scale

from threading import activeCount

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d
import sklearn.metrics
import math
from scipy.optimize import minimize
import numpy as np

In [17]:
def expected_improvement(X, X_sample, Y_sample, gpr, xi=0.01):
    '''
    Computes the expected_improvement at points X based on existing samples X_sample
    and Y_sample using a Gaussian process surrogate model.

    Args:
        X: Points at which expected_improvement shall be computed (m x d).
        X_sample: Sample locations (n x d).
        Y_sample: Sample values (n x 1).
        gpr: A GaussianProcessRegressor fitted to samples.
        xi: Exploitation-exploration trade-off parameter.

    Returns:
        Expected improvements at points X.
    '''
    mu, sigma = gpr.predict(X, return_std=True)
    mu_sample = gpr.predict(X_sample)

    sigma = sigma.reshape(-1, 1)

    # Needed for noise-based model,
    # otherwise use np.max(Y_sample).
    # See also section 2.4 in [...]
    mu_sample_opt = np.max(mu_sample)

    with np.errstate(divide='warn'):
        imp = mu - mu_sample_opt - xi
        Z = imp / sigma
        ei = imp * norm.cdf(Z) + sigma * norm.pdf(Z)
        ei[sigma == 0.0] = 0.0

    return ei

def Upper_boundary(X, X_sample, Y_sample, gpr, sigma):

    output1, output2 = gpr.predict(X, return_std=True)
    output1 = output1.flatten()
    prediction = np.maximum(output1,output2)
    std = np.minimum(output1,output2)


    upper_bound_value = np.add(prediction, sigma*std)

    return upper_bound_value

def propose_location(acquisition, X_sample, Y_sample, gpr, bounds, sigma = 1, n_restarts=25):
    '''
    Proposes the next sampling point by optimizing the acquisition function.

    Args:
        acquisition: Acquisition function.
        X_sample: Sample locations (n x d).
        Y_sample: Sample values (n x 1).
        gpr: A GaussianProcessRegressor fitted to samples.

    Returns:
        Location of the acquisition function maximum.
    '''
    dim = X_sample.shape[1]
    min_val = 1
    min_x = None

    def min_obj(X):
        # Minimization objective is the negative acquisition function

        return -acquisition(X.reshape(-1, dim), X_sample, Y_sample, gpr, sigma)

    # Find the best optimum by starting from n_restart different random points.
    for x0 in np.random.uniform(bounds[:, 0], bounds[:, 1], size=(n_restarts, dim)):
        res = minimize(min_obj, x0=x0, bounds=bounds, method='L-BFGS-B')
        if res.fun < min_val:
          min_val = res.fun
            #min_val = res.fun[0]
          min_x = res.x

    return min_x.reshape(-1, 1)

def distance(p1, p2):
    return math.sqrt((p2[0] - p1[0])**2 + (p2[1] - p1[1])**2)
def closest(pt, others):
  # return closest point to pt in others
    return min(others, key = lambda i: distance(pt, i))

def denormalize(X_std):
    ratio = [3, 20]
    min = [1, 15]
    X_original = X_std * ratio + min
    return X_original
def Normalize(X):
    ratio = [3, 20]
    min = [1, 15]
    X_nml = (X - min) / ratio
    return X_nml

def predict(gpr, prediction_history, std_history, i, X_init, Y_init, X_pool, Y_pool, X_next, X_next_availble, bounds, X_normalized = True, All_samples_at_once = True, Acquisition_function = Upper_boundary):

    if i == 0:
        prediction = []
        # Normalize
        if X_normalized:
            if len(X_init) != 0:
                X_init = Normalize(X_init)
            if len(X_pool) != 0:
                X_pool = Normalize(X_pool)
        # Put all samples to init
        if All_samples_at_once:
            X_init = np.vstack((X_init, X_pool))
            Y_init = np.vstack((Y_init, Y_pool))


        # Initialize samples
        global X_sample
        X_sample = X_init
        global Y_sample
        Y_sample = Y_init

        plt.figure(figsize=(12, 10 * 3))
        plt.subplots_adjust(hspace=0.4)


        if X_normalized:
            bounds = np.array([[0,1],[0,1]])
        else:
            bounds = np.array([[1,4],[15,35]])

    # Update Gaussian process with existing samples
    gpr.fit(X_sample, Y_sample)


    xi = 0.01 #调这儿来改expected_improvement的参数
    sigma = 10 #调这儿来改Upper_boundary的参数 0 1 10 均值+std*sigma 会改变std权重


    # Two methods to propose next sample location:
    # Find the sample that is closest to proposed location
    if Acquisition_function == Upper_boundary:
        X_proposal = propose_location(Upper_boundary, X_sample, Y_sample, gpr, bounds, sigma = sigma).flatten()
    if Acquisition_function == expected_improvement:
        X_proposal = propose_location(expected_improvement, X_sample, Y_sample, gpr, bounds, xi).flatten()

    X_next_availble_closest_to_proposeal = closest(X_proposal, X_pool)

    # Find the sample in pool with the highest upper_bound_value
    if Acquisition_function == Upper_boundary:
        aquisition_function_value = Upper_boundary(X_pool, X_sample, Y_sample, gpr, sigma = sigma)
        index = np.where(aquisition_function_value == np.amax(aquisition_function_value))
        X_next_availble_with_highest_aquisition_function_value = X_pool[index][0].reshape(1,2)[0]
    if Acquisition_function == expected_improvement:
        aquisition_function_value = expected_improvement(X_pool, X_sample, Y_sample, gpr, xi)
        index = np.where(aquisition_function_value == np.amax(aquisition_function_value))
        X_next_availble_with_highest_aquisition_function_value = X_pool[index][0]
    if (Acquisition_function != Upper_boundary) & (Acquisition_function != expected_improvement):
        print('WARNING: Acquisition_function not sepcified!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')


    # Choose which method to use:
    X_next_availble = X_next_availble_closest_to_proposeal#调这儿来改用那个method找pool里的下一个最高点。 X_next_availble_closest_to_proposeal or X_next_availble_with_highest_aquisition_function_value



    # Obtain Y_next_availble using X_next_availble
    t1 = np.argwhere(X_pool[:,0] == X_next_availble[0])
    t2 = np.argwhere(X_pool[:,1] == X_next_availble[1])
    index = np.intersect1d(t1,t2)
    Y_next_availble = Y_pool[index]

    # Record prediction_history
    prediction_next,std1 = gpr.predict(X_proposal.reshape(1,2),return_std=True)
    prediction,std = gpr.predict(X_next_availble.reshape(1,2),return_std=True)
    print('prediction_next is',prediction_next,'std1 is',std1)
    print('prediction is',prediction,'std1 is',std)
    prediction_history.append(float(prediction))
    std_history.append(float(std))


    #Denormalize to original datas for plotting
    if X_normalized:
        X_sample_dn = denormalize(X_sample)
        X_next_availble_dn = denormalize(X_next_availble)
        X_proposal_dn = denormalize(X_proposal)
    else:
        X_sample_dn = X_sample
        X_next_availble_dn = X_next_availble
        X_proposal_dn = X_proposal


    print("the next proposed location is {}，predicted to be {},std to be {}".format(X_proposal_dn, prediction_next,std1))
    print(" ")
    print("the next sample going to be used is {}, predicted to be {},std to be {}".format(X_next_availble_dn, prediction,std))
    print(" ")
    print("Predition history: ")
    print(prediction_history)
    print(" ")
    print("std history: ")
    print(std_history)
    print(" ")
    print("X's used: ")
    print(X_sample_dn)
    print(" ")
    print("Y's used: ")
    print(Y_sample)
    print(" ")
    print("xi is {}".format(xi))
    print(" ")


    # Plot 3D pridictions
    fig = plt.figure(figsize=(10,6))
    ax1 = fig.add_subplot(111, projection='3d')

    x1 = np.arange(0.9,4.2,0.1)
    x2 = np.arange(14,36,0.5)
    X1,X2 = np.meshgrid(x1,x2)

    test1 = np.reshape(X1,(1452,1))
    test2 = np.reshape(X2,(1452,1))
    X12 = np.concatenate((test1, test2), axis=1)
    if X_normalized:
        Y = gpr.predict(Normalize(X12))
    else:
        Y = gpr.predict(X12)
    Y = np.reshape(Y,(44, 33))


    mycmap = plt.get_cmap('gist_earth')
    ax1.set_title('Iteration {}'.format(i))
    surf1 = ax1.plot_surface(X1, X2, Y, cmap=mycmap)
    ax1.scatter(X_sample_dn[:,0], X_sample_dn[:,1], Y_sample[:], color = 'black')
    ax1.scatter(X_proposal_dn[0], X_proposal_dn[1], Y_sample[-1], color='yellow')
    ax1.scatter(X_next_availble_dn[0], X_next_availble_dn[1], Y_sample[-1], color='red')
    fig.colorbar(surf1, ax=ax1, shrink=0.5, aspect=5)
    plt.show()

    plot = plt.pcolormesh(X1, X2, Y, cmap='RdBu', shading='auto')

    # Plot 2D pridictions
    cset = plt.contour(X1, X2, Y, cmap='gray', levels = np.arange(95, 103, 0.2))#改contour的levels
    plt.clabel(cset, inline=True)
    plt.plot(X_sample_dn[:,0], X_sample_dn[:,1], 'o', color='black')
    plt.plot(X_proposal_dn[0], X_proposal_dn[1], 'o', color='yellow')
    plt.plot(X_next_availble_dn[0], X_next_availble_dn[1], 'o', color='red')
    plt.colorbar(plot)
    plt.show()

    # Add new sample to previous samples
    X_sample = np.vstack((X_sample, X_next_availble))
    Y_sample = np.vstack((Y_sample, Y_next_availble))

    # Remove used sample
    index_to_remove = np.where(np.all(X_pool == X_next_availble,axis=1))
    X_pool = np.delete(X_pool, index_to_remove, 0)
    Y_pool = np.delete(Y_pool, index_to_remove, 0)

    i +=1

    print('i is',i)

    return gpr, prediction_history, std_history, i, X_init, Y_init, X_pool, Y_pool, X_next, X_next_availble, bounds, denormalize(X_sample)

def rmse(y_test,y_prediction):
  MSE = sklearn.metrics.mean_squared_error(y_test,y_prediction)
  RMSE = math.sqrt(MSE)
  return RMSE


#Data Setup

In [18]:
X_train_6 = np.array([[1,15], [1,20], [1,25], [1,30], [1,35],
                  [2,15], [2,20], [2,25], [2,30], [2,35],
                  [3,15], [3,20], [3,25], [3,30], [3,35],
                  [4,15], [4,20], [4,25], [4,30], [4,35],
                  [1,15], [1,20], [1,25], [1,30], [1,35],
                  [2,15], [2,20], [2,25], [2,30], [2,35],
                  [3,15], [3,20], [3,25], [3,30], [3,35],
                  [4,15], [4,20], [4,25], [4,30], [4,35]])
y_train_6 = np.array([[99.39], [98.603], [97.8], [97.14], [96.65],
                  [99.54], [99.8], [99.94], [99.85], [99.93],
                  [98.62], [99.27], [99.26], [99.67], [99.51],
                  [97.75], [98.82], [99.07], [99.04], [99.15],
                  [96.77], [97.91], [98], [97.31], [97.41],
                  [99.63], [99.87], [99.89], [99.81], [99.82],
                  [99.17], [98.97], [99.72], [99.73], [99.82],
                  [99.02], [99.47], [99.71], [99.41], [99.55]])

std_train = np.array([[1.31], [0.3465], [0.1], [0.085], [0.38],
            [0.045], [0.035], [0.025], [0.02], [0.055],
            [0.275], [0.15], [0.23], [0.03], [0.155],
            [0.635], [0.325], [0.32], [0.185], [0.2]])

X_test_6 = np.array([[1.3,27.5], [2,17.5], [2.7,27.5], [2.5,35], [2.5,20], [1.5,35], [1.5,20], [2,27.5]])
y_test_6 = np.array([[99.045], [99.53], [99.54], [99.775], [99.535], [99.425], [99.69], [99.78]]) #Validation
std_test = np.array([[0.035], [0.13], [0.28], [0.105], [0.255], [0.105], [0.07], [0.1]])

X_normalized = True

#1D RQ with alpha

In [19]:
def change_model_1d_length_scale_alpha(X_train, X_test, y_train, y_test, std_test, alpha, length_scale, X_normalized, test_results, eval_metric):
    # initialize
    i = 0
    prediction_history = []
    std_history = []
    X_next = []
    X_next_availble = []

    if X_normalized:
        bounds = np.array([[0,1],[0,1]])     #True
    else:
        bounds = np.array([[1,4],[15,35]])    #False


    kernel = 1 * RQ(length_scale, alpha = alpha) # Change nu here if using Matern

    # GPR
    gpr = GaussianProcessRegressor(kernel=kernel, random_state=9)
    Acquisition_function = Upper_boundary #Change Acquisition_function between Upper_boundary and expected_improvement

    # Train
    model, prediction_history, std_history, i, X_init, Y_init, X_pool, Y_pool, X_next, X_next_availble, bounds, X_sample_dn = predict(gpr, prediction_history, std_history, i, X_train[0:4], y_train[0:4], X_train[4:], y_train[4:], X_next, X_next_availble, bounds, X_normalized=X_normalized, All_samples_at_once = 1, Acquisition_function = Acquisition_function)

    # Predict
    if X_normalized:
        X_test = Normalize(X_test)
    y_result = gpr.predict(X_test)
    output1, output2 = gpr.predict(X_test, return_std=True)
    output1 = output1.flatten()
    y_prediction = np.maximum(output1,output2)
    y_std = np.minimum(output1,output2)


    test_results[0]=(X_test)
    test_results[1]=(y_prediction)
    test_results[2]=(y_std)
    print('test_results')
    print(test_results)

    #Error Calculation
    if eval_metric == 'rmse':
      error_value_pred = rmse(y_test,y_prediction)
      error_value_std = rmse(std_test,y_std)
      likelihood_value = 0
    if eval_metric == 'likelihood':
      error_value_pred = 0
      error_value_std = 0
      likelihood_value = model.log_marginal_likelihood()

    return error_value_pred, error_value_std, likelihood_value, y_std, y_prediction

In [20]:
def grid_search_1d_length_scale_alpha(eval_metric, X_normalized, length_scale_start = 0.1, length_scale_end = 1.1, length_scale_step = 0.1, alpha_start = 1, alpha_end = 10, alpha_step = 1):
    graph_x = []
    graph_y = []
    graph_pred = []
    graph_std = []
    graph_likelihood = []
    test_results = [[],[],[]]

    for length_scale in np.arange(length_scale_start, length_scale_end, length_scale_step):#调这儿来改grid search的range
        graph_x_row = []
        graph_y_row = []
        graph_z_pred = []
        graph_z_std = []
        graph_z_likelihood = []

        for alpha in np.arange(alpha_start, alpha_end, alpha_step):
            print('Running %.4f %.4f' %(length_scale, alpha))
            error_value_pred, error_value_std, likelihood_value, y_std, y_prediction = change_model_1d_length_scale_alpha(X_train_6, X_test_6, y_train_6, y_test_6, std_test, alpha, length_scale, X_normalized, test_results, eval_metric)

            if eval_metric == 'likelihood':
                print('length_scale is', length_scale, 'alpha is', alpha, 'likelihood_value is', likelihood_value)
            else:
                print('length_scale is', length_scale, 'alpha is', alpha, 'error_value_pred is', error_value_pred, 'error_value_std is', error_value_std)

            graph_x_row.append(length_scale)
            graph_y_row.append(alpha)
            graph_z_pred.append(error_value_pred)
            graph_z_std.append(error_value_std)
            graph_z_likelihood.append(likelihood_value)

        graph_x.append(graph_x_row)
        graph_y.append(graph_y_row)
        graph_pred.append(graph_z_pred)
        graph_std.append(graph_z_std)
        graph_likelihood.append(graph_z_likelihood)
        print('')

    graph_x = np.array(graph_x)
    graph_y = np.array(graph_y)
    graph_pred = np.array(graph_pred)
    graph_std = np.array(graph_std)
    graph_likelihood = np.array(graph_likelihood)

    min_pred = np.min(graph_pred)
    min_std = np.min(graph_std)
    max_likelihood = np.max(graph_likelihood)

    pos_min_pred = np.argwhere(graph_pred == np.min(graph_pred))
    pos_min_std = np.argwhere(graph_std == np.min(graph_std))
    pos_max_likelihood = np.argwhere(graph_likelihood == np.max(graph_likelihood))

    graph_x_f = graph_x.flatten()
    graph_y_f = graph_y.flatten()
    graph_pred_f = graph_pred.flatten()
    graph_std_f = graph_std.flatten()
    graph_likelihood_f = graph_likelihood.flatten()

    graph_pred_f = minmax_scale(graph_pred_f,(5,30))
    graph_std_f = minmax_scale(graph_std_f,(5,30))
    graph_likelihood_f = minmax_scale(graph_std_f,(5,30))

    if eval_metric == 'likelihood':
        for i in range(len(graph_likelihood_f)):
            #print('ploting {}'.format(i))
            plt.plot(graph_x_f[i], graph_y_f[i], 'o', markersize = graph_likelihood_f[i], color='orange')

        plt.plot(graph_x[pos_max_likelihood[0][0],pos_max_likelihood[0][1]], graph_y[pos_max_likelihood[0][0],pos_max_likelihood[0][1]], 'o', markersize = 5, color='red');
        plt.title('likelihood w.r.t. different length scale and alpha')
        plt.show()
        print('With likelihood as metric, the maximum likelihood value is', max_likelihood, 'when kernel is RQ with 1d length scale of', graph_x[pos_max_likelihood[0][0],pos_max_likelihood[0][1]], 'and alpha of', graph_y[pos_max_likelihood[0][0],pos_max_likelihood[0][1]])

    if eval_metric == 'rmse':
        for i in range(len(graph_pred_f)):
            #print('ploting {}'.format(i))
            plt.plot(graph_x_f[i], graph_y_f[i], 'o', markersize = graph_pred_f[i], color='orange');

        plt.plot(graph_x[pos_min_pred[0][0],pos_min_pred[0][1]], graph_y[pos_min_pred[0][0],pos_min_pred[0][1]], 'o', markersize = 5, color='red');
        plt.title('rmse w.r.t. different length scale and alpha on prediction')
        plt.show()
        print('With rmse as metric, the minimum rmse on prediction is', min_pred, 'when kernel is RQ with 1d length scale of', graph_x[pos_min_pred[0][0],pos_min_pred[0][1]], 'and alpha of', graph_y[pos_min_pred[0][0],pos_min_pred[0][1]])


        for i in range(len(graph_std_f)):
            #print('ploting {}'.format(i))
            plt.plot(graph_x_f[i], graph_y_f[i], 'o', markersize = graph_std_f[i], color='orange');

        plt.plot(graph_x[pos_min_std[0][0],pos_min_std[0][1]], graph_y[pos_min_std[0][0],pos_min_std[0][1]], 'o', markersize = 5, color='red');
        plt.title('rmse w.r.t. different length scale and alpha on std')
        plt.show()
        print('With rmse as metric, the minimum rmse on std is', min_std, 'when kernel is RQ with 1d length scale of', graph_x[pos_min_std[0][0],pos_min_std[0][1]], 'and alpha of', graph_y[pos_min_std[0][0],pos_min_std[0][1]])



    #return graph_x,graph_y,graph_pred,graph_std

In [ ]:
#              rmse,likelihood
grid_search_1d_length_scale_alpha('rmse', X_normalized, 0.1, 1.1, 0.1, 1, 10, 1)

#1D Matern with nu

In [22]:
def change_model_1d_length_scale_nu(X_train, X_test, y_train, y_test, std_test, nu, length_scale, X_normalized, test_results, eval_metric):
    # initialize
    i = 0
    prediction_history = []
    std_history = []
    X_next = []
    X_next_availble = []

    if X_normalized:
        bounds = np.array([[0,1],[0,1]])     #True
    else:
        bounds = np.array([[1,4],[15,35]])    #False


    kernel = 1 * Matern(length_scale = length_scale, nu = nu) # Change nu here if using Matern

    # GPR
    gpr = GaussianProcessRegressor(kernel=kernel, random_state=9)
    Acquisition_function = Upper_boundary #Change Acquisition_function between Upper_boundary and expected_improvement

    # Train
    model, prediction_history, std_history, i, X_init, Y_init, X_pool, Y_pool, X_next, X_next_availble, bounds, X_sample_dn = predict(gpr, prediction_history, std_history, i, X_train[0:4], y_train[0:4], X_train[4:], y_train[4:], X_next, X_next_availble, bounds, X_normalized=X_normalized, All_samples_at_once = 1, Acquisition_function = Acquisition_function)

    # Predict
    if X_normalized:
        X_test = Normalize(X_test)
    y_result = gpr.predict(X_test)
    output1, output2 = gpr.predict(X_test, return_std=True)
    output1 = output1.flatten()
    y_prediction = np.maximum(output1,output2)
    y_std = np.minimum(output1,output2)


    test_results[0]=(X_test)
    test_results[1]=(y_prediction)
    test_results[2]=(y_std)
    print('test_results')
    print(test_results)

    #Error Calculation
    if eval_metric == 'rmse':
      error_value_pred = rmse(y_test,y_prediction)
      error_value_std = rmse(std_test,y_std)
      likelihood_value = 0
    if eval_metric == 'likelihood':
      error_value_pred = 0
      error_value_std = 0
      likelihood_value = model.log_marginal_likelihood()

    return error_value_pred, error_value_std, likelihood_value, y_std, y_prediction

In [23]:
def grid_search_1d_length_scale_nu(eval_metric, X_normalized, length_scale_start = 0.1, length_scale_end = 1.1, length_scale_step = 0.1):
    graph_x = []
    graph_y = []
    graph_pred = []
    graph_std = []
    graph_likelihood = []
    test_results = [[],[],[]]
    nu_list = list((0.5, 1.5, 2.5, float('inf')))

    for length_scale in np.arange(length_scale_start, length_scale_end, length_scale_step):#调这儿来改grid search的range
        graph_x_row = []
        graph_y_row = []
        graph_z_pred = []
        graph_z_std = []
        graph_z_likelihood = []

        for nu in nu_list:
            print('Running %.4f %.4f' %(length_scale, nu))
            error_value_pred, error_value_std, likelihood_value, y_std, y_prediction = change_model_1d_length_scale_nu(X_train_6, X_test_6, y_train_6, y_test_6, std_test, nu, length_scale, X_normalized, test_results, eval_metric)

            if eval_metric == 'likelihood':
                print('length_scale is', length_scale, 'nu is', nu, 'likelihood_value is', likelihood_value)
            else:
                print('length_scale is', length_scale, 'nu is', nu, 'error_value_pred is', error_value_pred, 'error_value_std is', error_value_std)

            graph_x_row.append(length_scale)
            graph_y_row.append(nu)
            graph_z_pred.append(error_value_pred)
            graph_z_std.append(error_value_std)
            graph_z_likelihood.append(likelihood_value)

        graph_x.append(graph_x_row)
        graph_y.append(graph_y_row)
        graph_pred.append(graph_z_pred)
        graph_std.append(graph_z_std)
        graph_likelihood.append(graph_z_likelihood)
        print('')

    graph_x = np.array(graph_x)
    graph_y = np.array(graph_y)
    graph_pred = np.array(graph_pred)
    graph_std = np.array(graph_std)
    graph_likelihood = np.array(graph_likelihood)

    min_pred = np.min(graph_pred)
    min_std = np.min(graph_std)
    max_likelihood = np.max(graph_likelihood)

    pos_min_pred = np.argwhere(graph_pred == np.min(graph_pred))
    pos_min_std = np.argwhere(graph_std == np.min(graph_std))
    pos_max_likelihood = np.argwhere(graph_likelihood == np.max(graph_likelihood))

    graph_x_f = graph_x.flatten()
    graph_y_f = graph_y.flatten()
    graph_pred_f = graph_pred.flatten()
    graph_std_f = graph_std.flatten()
    graph_likelihood_f = graph_likelihood.flatten()

    graph_pred_f = minmax_scale(graph_pred_f,(5,30))
    graph_std_f = minmax_scale(graph_std_f,(5,30))
    graph_likelihood_f = minmax_scale(graph_std_f,(5,30))

    if eval_metric == 'likelihood':
        for i in range(len(graph_likelihood_f)):
            #print('ploting {}'.format(i))
            plt.plot(graph_x_f[i], graph_y_f[i], 'o', markersize = graph_likelihood_f[i], color='orange')

        plt.plot(graph_x[pos_max_likelihood[0][0],pos_max_likelihood[0][1]], graph_y[pos_max_likelihood[0][0],pos_max_likelihood[0][1]], 'o', markersize = 5, color='red');
        plt.title('likelihood w.r.t. different length scale and nu')
        plt.show()
        print('With likelihood as metric, the maximum likelihood value is', max_likelihood, 'when kernel is Matern with 1d length scale of', graph_x[pos_max_likelihood[0][0],pos_max_likelihood[0][1]], 'and nu of', nu_list[pos_max_likelihood[0][1]])

    if eval_metric == 'rmse':
        for i in range(len(graph_pred_f)):
            #print('ploting {}'.format(i))
            plt.plot(graph_x_f[i], graph_y_f[i], 'o', markersize = graph_pred_f[i], color='orange');

        plt.plot(graph_x[pos_min_pred[0][0],pos_min_pred[0][1]], graph_y[pos_min_pred[0][0],pos_min_pred[0][1]], 'o', markersize = 5, color='red');
        plt.title('rmse w.r.t. different length scale and nu on prediction')
        plt.show()
        print('With rmse as metric, the minimum rmse on prediction is', min_pred, 'when kernel is Matern with 1d length scale of', graph_x[pos_min_pred[0][0],pos_min_pred[0][1]], 'and nu of', nu_list[pos_min_pred[0][1]])


        for i in range(len(graph_std_f)):
            #print('ploting {}'.format(i))
            plt.plot(graph_x_f[i], graph_y_f[i], 'o', markersize = graph_std_f[i], color='orange');

        plt.plot(graph_x[pos_min_std[0][0],pos_min_std[0][1]], graph_y[pos_min_std[0][0],pos_min_std[0][1]], 'o', markersize = 5, color='red');
        plt.title('rmse w.r.t. different length scale and nu on std')
        plt.show()
        print('With rmse as metric, the minimum rmse on std is', min_std, 'when kernel is Matern with 1d length scale of', graph_x[pos_min_std[0][0],pos_min_std[0][1]], 'and nu of', nu_list[pos_min_std[0][1]])



    #return graph_x,graph_y,graph_pred,graph_std

In [ ]:
#              rmse,likelihood
grid_search_1d_length_scale_nu('likelihood', X_normalized, 0.1, 1.1, 0.1)

nufloat, default=1.5

The parameter nu controlling the smoothness of the learned function. The smaller nu, the less smooth the approximated function is. For nu=inf, the kernel becomes equivalent to the RBF kernel and for nu=0.5 to the absolute exponential kernel. Important intermediate values are nu=1.5 (once differentiable functions) and nu=2.5 (twice differentiable functions). Note that values of nu not in [0.5, 1.5, 2.5, inf] incur a considerably higher computational cost (appr. 10 times higher) since they require to evaluate the modified Bessel function. Furthermore, in contrast to l, nu is kept fixed to its initial value and not optimized.

#1D LengthScale Search

In [25]:
def change_model_1d_length_scale(X_train, X_test, y_train, y_test, std_test, kernel_name, length_scale, X_normalized, test_results, eval_metric):
    # initialize
    i = 0
    prediction_history = []
    std_history = []
    X_next = []
    X_next_availble = []

    if X_normalized:
        bounds = np.array([[0,1],[0,1]])     #True
    else:
        bounds = np.array([[1,4],[15,35]])    #False

    if kernel_name == 'RBF':
      kernel = 1 * RBF(length_scale)
    if kernel_name == 'RQ':
      kernel = 1 * RQ(length_scale, alpha = 50) # Change alpha here if using RQ kernel
    if kernel_name == 'Matern':
      kernel = 1 * Matern(length_scale, nu = 18) # Change nu here if using Matern
    if kernel_name == 'Exp':
      kernel = Exp(length_scale = 1, periodicity=1)
    # GPR
    gpr = GaussianProcessRegressor(kernel=kernel, random_state=9)
    Acquisition_function = Upper_boundary #Change Acquisition_function between Upper_boundary and expected_improvement

    # Train
    model, prediction_history, std_history, i, X_init, Y_init, X_pool, Y_pool, X_next, X_next_availble, bounds, X_sample_dn = predict(gpr, prediction_history, std_history, i, X_train[0:4], y_train[0:4], X_train[4:], y_train[4:], X_next, X_next_availble, bounds, X_normalized=X_normalized, All_samples_at_once = 1, Acquisition_function = Acquisition_function)

    # Predict
    if X_normalized:
        X_test = Normalize(X_test)
    y_result = gpr.predict(X_test)
    output1, output2 = gpr.predict(X_test, return_std=True)
    output1 = output1.flatten()
    y_prediction = np.maximum(output1,output2)
    y_std = np.minimum(output1,output2)


    test_results[0]=(X_test)
    test_results[1]=(y_prediction)
    test_results[2]=(y_std)
    print('test_results')
    print(test_results)

    #Error Calculation
    if eval_metric == 'rmse':
      error_value_pred = rmse(y_test,y_prediction)
      error_value_std = rmse(std_test,y_std)
      likelihood_value = 0
    if eval_metric == 'likelihood':
      error_value_pred = 0
      error_value_std = 0
      likelihood_value = model.log_marginal_likelihood()

    return error_value_pred, error_value_std, likelihood_value, y_std, y_prediction

In [26]:
def grid_search_1d_length_scale(kernel_name, eval_metric, X_normalized, length_scale_start = 0.1, length_scale_end = 1.1, length_scale_step = 0.1):
    graph_x = []
    graph_pred = []
    graph_std = []
    graph_likelihood = []
    test_results = [[],[],[]]

    for length_scale in np.arange(length_scale_start, length_scale_end, length_scale_step):#调这儿来改grid search的range
        print('Running %.4f' %(length_scale))
        error_value_pred, error_value_std, likelihood_value, y_std, y_prediction = change_model_1d_length_scale(X_train_6, X_test_6, y_train_6, y_test_6, std_test, kernel_name, length_scale, X_normalized, test_results, eval_metric)
        if eval_metric == 'likelihood':
            print('length_scale is', length_scale, 'likelihood_value is', likelihood_value)
        else:
            print('length_scale is', length_scale, 'error_value_pred is', error_value_pred, 'error_value_std is', error_value_std)
        graph_x.append(length_scale)
        graph_pred.append(error_value_pred)
        graph_std.append(error_value_std)
        graph_likelihood.append(likelihood_value)
        print('')
    graph_x=np.array(graph_x)
    graph_pred=np.array(graph_pred)
    graph_std=np.array(graph_std)
    graph_likelihood = np.array(graph_likelihood)

    min_pred = np.min(graph_pred)
    min_std = np.min(graph_std)
    max_likelihood = np.max(graph_likelihood)

    pos_min_pred = np.argwhere(graph_pred == np.min(graph_pred))[0]
    pos_min_std = np.argwhere(graph_std == np.min(graph_std))[0]
    pos_max_likelihood = np.argwhere(graph_likelihood == np.max(graph_likelihood))[0]

    graph_x_f = graph_x.flatten()
    graph_pred_f = graph_pred.flatten()
    graph_std_f = graph_std.flatten()
    graph_likelihood_f = graph_likelihood.flatten()

    graph_pred_f = minmax_scale(graph_pred_f,(5,30))
    graph_std_f = minmax_scale(graph_std_f,(5,30))
    graph_likelihood_f = minmax_scale(graph_std_f,(5,30))


    if eval_metric == 'likelihood':
        plt.scatter(graph_x, graph_likelihood, color='orange')
        plt.scatter(graph_x[pos_max_likelihood], graph_likelihood[pos_max_likelihood], color='red')
        plt.title('likelihood w.r.t. different length scale')
        plt.show()
        print('With likelihood as metric, the maximum likelihood value is', max_likelihood, 'when kernel is', kernel_name, 'with 1d length scale of', graph_x[pos_max_likelihood[0]])

    if eval_metric == 'rmse':
        plt.scatter(graph_x, graph_pred, color='orange')
        plt.scatter(graph_x[pos_min_pred], graph_pred[pos_min_pred], color='red')
        plt.title('rmse w.r.t. different length scale on prediction')
        plt.show()
        print('With rmse as metric, the minimum rmse on prediction is', min_pred, 'when kernel is', kernel_name, 'with 1d length scale of', graph_x[pos_min_pred[0]])


        plt.scatter(graph_x, graph_std, color='orange')
        plt.scatter(graph_x[pos_min_std], graph_std[pos_min_std], color='red')
        plt.title('rmse w.r.t. different length scale on std')
        plt.show()
        print('With rmse as metric, the minimum rmse on std is', min_std, 'when kernel is', kernel_name, 'with 1d length scale of', graph_x[pos_min_std[0]])


    #return graph_x,graph_pred,graph_std

In [ ]:
#RBF, Matern, RQ;     rmse,likelihood
grid_search_1d_length_scale('RBF', 'likelihood', X_normalized, length_scale_start = 0.1, length_scale_end = 1.1, length_scale_step = 0.1)

In [ ]:
X_normalized = True
test_results = [[],[],[]]
length_scale = 0.4
error_value_pred, error_value_std, likelihood_value, y_std, y_prediction = change_model_1d_length_scale(X_train_6, X_test_6, y_train_6, y_test_6, std_test, 'RBF', length_scale, X_normalized, test_results, 'likelihood')

#2D LengthScale Search on Matern(fixed nu = 1.5) and RBF

In [29]:
def change_model_2d_length_scale(X_train, X_test, y_train, y_test, std_test, kernel_name, length_scale, X_normalized, test_results, eval_metric):
    # initialize
    i = 0
    prediction_history = []
    std_history = []
    X_next = []
    X_next_availble = []

    if X_normalized:
        bounds = np.array([[0,1],[0,1]])     #True
    else:
        bounds = np.array([[1,4],[15,35]])    #False

    if kernel_name == 'RBF':
      kernel = 1 * RBF(length_scale)
    if kernel_name == 'RQ':
      kernel = 1 * RQ(length_scale, alpha = 50) # Change alpha here if using RQ kernel
    if kernel_name == 'Matern':
      kernel = 1 * Matern(length_scale, nu = 18) # Change nu here if using Matern
    if kernel_name == 'Exp':
      kernel = Exp(length_scale = 1, periodicity=1)
    # GPR
    gpr = GaussianProcessRegressor(kernel=kernel, random_state=9)
    Acquisition_function = Upper_boundary #Change Acquisition_function between Upper_boundary and expected_improvement

    # Train
    model, prediction_history, std_history, i, X_init, Y_init, X_pool, Y_pool, X_next, X_next_availble, bounds, X_sample_dn = predict(gpr, prediction_history, std_history, i, X_train[0:4], y_train[0:4], X_train[4:], y_train[4:], X_next, X_next_availble, bounds, X_normalized=X_normalized, All_samples_at_once = 1, Acquisition_function = Acquisition_function)

    # Predict
    if X_normalized:
        X_test = Normalize(X_test)
    y_result = gpr.predict(X_test)
    output1, output2 = gpr.predict(X_test, return_std=True)
    output1 = output1.flatten()
    y_prediction = np.maximum(output1,output2)
    y_std = np.minimum(output1,output2)


    test_results[0]=(X_test)
    test_results[1]=(y_prediction)
    test_results[2]=(y_std)
    print('test_results')
    print(test_results)

    #Error Calculation
    if eval_metric == 'rmse':
      error_value_pred = rmse(y_test,y_prediction)
      error_value_std = rmse(std_test,y_std)
      likelihood_value = 0
    if eval_metric == 'likelihood':
      error_value_pred = 0
      error_value_std = 0
      likelihood_value = model.log_marginal_likelihood()

    return error_value_pred, error_value_std, likelihood_value, y_std, y_prediction

In [30]:
def grid_search_2d_length_scale(kernel_name, eval_metric, X_normalized, length_scale_0_start = 0.6, length_scale_0_end = 1.1, length_scale_0_step = 0.1, length_scale_1_start = 0.2, length_scale_1_end = 1.1, length_scale_1_step = 0.1):
    graph_x = []
    graph_y = []
    graph_pred = []
    graph_std = []
    graph_likelihood = []
    test_results = [[],[],[]]

    for length_scale_0 in np.arange(length_scale_0_start, length_scale_0_end, length_scale_0_step):#调这儿来改grid search的range
        graph_x_row = []
        graph_y_row = []
        graph_z_pred = []
        graph_z_std = []
        graph_z_likelihood = []

        for length_scale_1 in np.arange(length_scale_1_start, length_scale_1_end, length_scale_1_step):
            print('Running %.4f %.4f' %(length_scale_0, length_scale_1))
            length_scale = [length_scale_0,length_scale_1]
            error_value_pred, error_value_std, likelihood_value, y_std, y_prediction = change_model_2d_length_scale(X_train_6, X_test_6, y_train_6, y_test_6, std_test, kernel_name, length_scale, X_normalized, test_results, eval_metric)


            if eval_metric == 'likelihood':
                print('length_scale_0 is', length_scale_0, 'length_scale_1 is', length_scale_1, 'likelihood_value is', likelihood_value)
            else:
                print('length_scale_0 is', length_scale_0, 'length_scale_1 is', length_scale_1, 'error_value_pred is', error_value_pred, 'error_value_std is', error_value_std)

            graph_x_row.append(length_scale_0)
            graph_y_row.append(length_scale_1)
            graph_z_pred.append(error_value_pred)
            graph_z_std.append(error_value_std)
            graph_z_likelihood.append(likelihood_value)

        graph_x.append(graph_x_row)
        graph_y.append(graph_y_row)
        graph_pred.append(graph_z_pred)
        graph_std.append(graph_z_std)
        graph_likelihood.append(graph_z_likelihood)
        print('')

    graph_x = np.array(graph_x)
    graph_y = np.array(graph_y)
    graph_pred = np.array(graph_pred)
    graph_std = np.array(graph_std)
    graph_likelihood = np.array(graph_likelihood)

    min_pred = np.min(graph_pred)
    min_std = np.min(graph_std)
    max_likelihood = np.max(graph_likelihood)

    pos_min_pred = np.argwhere(graph_pred == np.min(graph_pred))
    pos_min_std = np.argwhere(graph_std == np.min(graph_std))
    pos_max_likelihood = np.argwhere(graph_likelihood == np.max(graph_likelihood))

    graph_x_f = graph_x.flatten()
    graph_y_f = graph_y.flatten()
    graph_pred_f = graph_pred.flatten()
    graph_std_f = graph_std.flatten()
    graph_likelihood_f = graph_likelihood.flatten()

    graph_pred_f = minmax_scale(graph_pred_f,(5,30))
    graph_std_f = minmax_scale(graph_std_f,(5,30))
    graph_likelihood_f = minmax_scale(graph_std_f,(5,30))

    if eval_metric == 'likelihood':
        for i in range(len(graph_likelihood_f)):
            #print('ploting {}'.format(i))
            plt.plot(graph_x_f[i], graph_y_f[i], 'o', markersize = graph_likelihood_f[i], color='orange')

        plt.plot(graph_x[pos_max_likelihood[0][0],pos_max_likelihood[0][1]], graph_y[pos_max_likelihood[0][0],pos_max_likelihood[0][1]], 'o', markersize = 5, color='red');
        plt.title('likelihood w.r.t. different length scale')
        plt.show()
        print('With likelihood as metric, the maximum likelihood value is', max_likelihood, 'when kernel is', kernel_name, 'with 2d length scale of', graph_x[pos_max_likelihood[0][0],pos_max_likelihood[0][1]], 'and', graph_y[pos_max_likelihood[0][0],pos_max_likelihood[0][1]])

    if eval_metric == 'rmse':
        for i in range(len(graph_pred_f)):
            #print('ploting {}'.format(i))
            plt.plot(graph_x_f[i], graph_y_f[i], 'o', markersize = graph_pred_f[i], color='orange');

        plt.plot(graph_x[pos_min_pred[0][0],pos_min_pred[0][1]], graph_y[pos_min_pred[0][0],pos_min_pred[0][1]], 'o', markersize = 5, color='red');
        plt.title('rmse w.r.t. different length scale on prediction')
        plt.show()
        print('With rmse as metric, the minimum rmse on prediction is', min_pred, 'when kernel is', kernel_name, 'with 2d length scale of', graph_x[pos_min_pred[0][0],pos_min_pred[0][1]], 'and', graph_y[pos_min_pred[0][0],pos_min_pred[0][1]])


        for i in range(len(graph_std_f)):
            #print('ploting {}'.format(i))
            plt.plot(graph_x_f[i], graph_y_f[i], 'o', markersize = graph_std_f[i], color='orange');

        plt.plot(graph_x[pos_min_std[0][0],pos_min_std[0][1]], graph_y[pos_min_std[0][0],pos_min_std[0][1]], 'o', markersize = 5, color='red');
        plt.title('rmse w.r.t. different length scale on std')
        plt.show()
        print('With rmse as metric, the minimum rmse on std is', min_std, 'when kernel is', kernel_name, 'with 2d length scale of', graph_x[pos_min_std[0][0],pos_min_std[0][1]], 'and', graph_y[pos_min_std[0][0],pos_min_std[0][1]])



    #return graph_x,graph_y,graph_pred,graph_std

In [ ]:
#RBF, Matern, RQ;     rmse,likelihood
grid_search_2d_length_scale('Matern', 'rmse', X_normalized, 0.6, 1.1, 0.1, 0.2, 1.1, 0.1)

In [ ]:
X_normalized = True
test_results = [[],[],[]]
length_scale = [0.2, 0.7]
error_value_pred, error_value_std, likelihood_value, y_std, y_prediction = change_model_2d_length_scale(X_train_6, X_test_6, y_train_6, y_test_6, std_test, 'RBF', length_scale, X_normalized, test_results, 'likelihood')